In [1]:
# load dependencies

## first time series test
# append ../../ to make link to main repo work
import sys
sys.path.append('../../')
sys.path.append('/Applications/anaconda3/envs/PhD_clean/lib/python3.11/site-packages/')


# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex


import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
# import rioxarray
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

from scipy import spatial
# from scikit_spatial import Line as sksline
# import scipy.spatial.transform 

from skspatial.objects import Line as sksline
import scipy.spatial.transform 

import json
from shapely.geometry import Point, LineString, mapping
from functools import partial
from shapely.ops import transform

import sys
sys.executable


###########################
# import main local package
import SPOTSAR_main as sm



In [2]:
### steps:

# 1. load rasters
# 1.1 get dates from file
# 2. load lavaflows and craters
# 3. plot results in grid

In [14]:
from SPOTSAR_main.misc_func.read_line_shp import read_line_shp
from PIL import Image
# 1. load rasters

Data_dir  ="/Users/tz20896/Documents/PhD/projects/Merapi2021/S1/daisychain/rasters/*"
data_list =  np.sort(glob.glob(Data_dir))
file_list = [item[-29:] for item in np.sort(glob.glob(Data_dir))]
date1 = [file[0:8] for file in file_list]
date2 = [file[9:17] for file in file_list]

# data_list =  glob.glob(Data_dir)
print(date1)
print(date2)
print(data_list)

image_files = []

for file in data_list:
    print(file)
    with Image.open(file) as img:
        image_files.append(np.array(img))

# print(np.shape(image_files))
    


# fig, ax = plt.subplots(1,1)

# ax.imshow(np.array(test_img))

['20200112', '20200124', '20200205', '20200217', '20200229', '20200312', '20200324', '20200405', '20200417', '20200429', '20200511', '20200604', '20200616', '20200628', '20200815', '20200827', '20200908', '20200920', '20201002', '20201014', '20201026', '20201107', '20201201', '20201213', '20201225', '20210106', '20210118', '20210130']
['20200124', '20200205', '20200217', '20200229', '20200312', '20200324', '20200405', '20200417', '20200429', '20200511', '20200604', '20200616', '20200628', '20200815', '20200827', '20200908', '20200920', '20201002', '20201014', '20201026', '20201107', '20201201', '20201213', '20201225', '20210106', '20210118', '20210130', '20210211']
['/Users/tz20896/Documents/PhD/projects/Merapi2021/S1/daisychain/rasters/20200112_20200124.diff.gc.jpg'
 '/Users/tz20896/Documents/PhD/projects/Merapi2021/S1/daisychain/rasters/20200124_20200205.diff.gc.jpg'
 '/Users/tz20896/Documents/PhD/projects/Merapi2021/S1/daisychain/rasters/20200205_20200217.diff.gc.jpg'
 '/Users/tz208

In [2]:
from SPOTSAR_main.misc_func.read_line_shp import read_line_shp
from PIL import Image
# 1. load rasters

### get EDM positions
pos_bad = np.array([110.410626, -7.526175])
reflectors = np.array(
    [[110.439757, -7.537567], [110.440165, -7.537451], [110.439260, -7.537533]]
)

### lava flows and crater
L1888_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1888_v2.shp")
L1948_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1948.shp")
L1956_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1956.shp")
L1992_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1992.shp")
L1997_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1997.shp")
L1998_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1998.shp")
L1986_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1986.shp")
L1984_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1984.shp")
L1931_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/L1931.shp")

OLD_CRATER_FILE = ("/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/old_crater_rims.shp")
CRATER_FILE = "/Users/tz20896/Documents/PhD/projects/Merapi2021/merapi_maps/Merapi_crater.shp"

# Extract latitude and longitude into separate columns
coords_L1888       = read_line_shp(L1888_FILE )
coords_L1931       = read_line_shp(L1931_FILE )
coords_L1948       = read_line_shp(L1948_FILE )
coords_L1956       = read_line_shp(L1956_FILE )
coords_L1984       = read_line_shp(L1984_FILE )
coords_L1986       = read_line_shp(L1986_FILE )
coords_L1992       = read_line_shp(L1992_FILE )
coords_L1997       = read_line_shp(L1997_FILE )
coords_L1998       = read_line_shp(L1998_FILE )
coords_CRATER      = read_line_shp(CRATER_FILE)
coords_OLD_CRATER  = read_line_shp(OLD_CRATER_FILE)


# separate multi-line
coords_L1956_1 = coords_L1956[0][0]
coords_L1956_2 = coords_L1956[1][0]

coords_pas1_CRATER   = coords_OLD_CRATER[0]
coords_pas2_CRATER   = coords_OLD_CRATER[1]
coords_1822_CRATER   = coords_OLD_CRATER[2]
coords_1872_CRATER   = coords_OLD_CRATER[3]
coords_1904_1_CRATER = coords_OLD_CRATER[4]
coords_1904_2_CRATER = coords_OLD_CRATER[5]
coords_1932_CRATER   = coords_OLD_CRATER[6]
coords_1961_1_CRATER = coords_OLD_CRATER[7]
coords_1961_2_CRATER = coords_OLD_CRATER[8]



## DEM data
# DEM
Merapi_DEM_file = "/Users/tz20896/Documents/PhD/projects/Merapi2021/geo/TDX_Merapi_WGS84_5m.tif"
with rio.open(Merapi_DEM_file) as img:
    DEM = img.read(1,masked=True) # rasterio bands are indexed from 1
    # extract DEM extent
    DEM_extent=[img.bounds.left,img.bounds.right,img.bounds.bottom,img.bounds.top]
    print('Band1 has shape', DEM.shape)
    height = DEM.shape[0]
    width  = DEM.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(img.transform, rows, cols)
    lons_DEM = np.array(xs)
    lats_DEM = np.array(ys)


# DEM
Merapi_HS_file = "/Users/tz20896/Documents/PhD/projects/Merapi2021/geo/TDX_Merapi_WGS84_HS.tif"
with rio.open(Merapi_HS_file) as img:
    SHADING = img.read(1,masked=True) # rasterio bands are indexed from 1






## defien query points fro time series plot
Q_points = [(110.44595,-7.53895),
            (110.44364,-7.53659),
            (110.440165,-7.537451),
            (110.44000,-7.53831),
            (110.44322,-7.54022),
            (110.44280,-7.54211),
            (110.43365,-7.54146),
            (110.44493,-7.54096),
            (110.44611,-7.54102),
            (110.44894,-7.53674),
            ]


Band1 has shape (2028, 1672)


In [29]:
%matplotlib osx
import string
import matplotlib.patheffects as path_effects
plt.close('all')
## plot results
## 28 ifgs (not all are helpfull ignore first 8? --> 20 left --> 5x4 grid)

fig = plt.figure(figsize=(8,8))

gs = GridSpec(5, 4, figure=fig)

# allocating panels in gridspec
ax00 = fig.add_subplot(gs[0, 0]) # 
ax01 = fig.add_subplot(gs[0, 1]) # 
ax02 = fig.add_subplot(gs[0, 2]) # 
ax03 = fig.add_subplot(gs[0, 3]) # 
# ax04 = fig.add_subplot(gs[0, 4]) # 

ax10 = fig.add_subplot(gs[1, 0]) # 
ax11 = fig.add_subplot(gs[1, 1]) # 
ax12 = fig.add_subplot(gs[1, 2]) # 
ax13 = fig.add_subplot(gs[1, 3]) # 
# ax14 = fig.add_subplot(gs[1, 4]) # 

ax20 = fig.add_subplot(gs[2, 0]) # 
ax21 = fig.add_subplot(gs[2, 1]) # 
ax22 = fig.add_subplot(gs[2, 2]) # 
ax23 = fig.add_subplot(gs[2, 3]) # 
# ax24 = fig.add_subplot(gs[2, 4]) # 

ax30 = fig.add_subplot(gs[3, 0]) # 
ax31 = fig.add_subplot(gs[3, 1]) # 
ax32 = fig.add_subplot(gs[3, 2]) # 
ax33 = fig.add_subplot(gs[3, 3]) # 
# ax34 = fig.add_subplot(gs[3, 4]) # 


ax40 = fig.add_subplot(gs[4, 0]) # 
ax41 = fig.add_subplot(gs[4, 1]) # 
ax42 = fig.add_subplot(gs[4, 2]) # 
ax43 = fig.add_subplot(gs[4, 3]) # 

## plot ifgs
lon_lims = [110.427, 110.45]
lat_lims = [-7.55, -7.533]
labels_annot = string.ascii_uppercase[0:len(fig.axes)]
bbox = dict(fc="1.0",alpha=0.5)
for i, (ax,ifg,d1,d2,lab_annot) in enumerate(zip(fig.axes,image_files[8:],date1[8:],date2[8:],labels_annot)):
    print(i)
    ax.imshow(ifg[::10,::10],extent=DEM_extent)
    ax.plot(coords_L1888[:,0]  ,coords_L1888[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1888')
    ax.plot(coords_L1948[:,0]  ,coords_L1948[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1948')
    ax.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='white',linestyle='--',label='L1956')
    ax.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='white',linestyle='--',)
    ax.plot(coords_L1992[:,0]  ,coords_L1992[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1992')
    ax.plot(coords_L1997[:,0]  ,coords_L1997[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1997')
    ax.plot(coords_L1998[:,0]  ,coords_L1998[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1998')
    ax.plot(coords_L1986[:,0]  ,coords_L1986[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1986')
    ax.plot(coords_L1984[:,0]  ,coords_L1984[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1984')
    ax.plot(coords_L1931[:,0]  ,coords_L1931[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1931')
    ax.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')

    ax.annotate(lab_annot,xy=[0.02,0.90],xycoords='axes fraction',fontsize=16, bbox=bbox)
    text = ax.annotate(f'{d1}-\n{d2}',xy=[0.02,0.10],xycoords='axes fraction',fontsize=12,color='white',weight='bold')
    # text = a.annotate(t,xy=q_p, xycoords='data',xytext=(1.5, 1.5), textcoords='offset points', fontsize=12, color='white',weight='bold' )
    text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'),
                       path_effects.Normal()])
    
    ax.set_xlim(lon_lims)
    ax.set_ylim(lat_lims)
    ax.set_aspect('equal', 'box')
    ax.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower right'))
    ax.set_axis_off()

fig.tight_layout()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [21]:
## test to load geotiffs with python
Image.MAX_IMAGE_PIXELS = 391835235
## DEM data
# DEM
TSX_files = np.sort(glob.glob("/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/1_day_tiffs/*.gc.tif"))
file_list = [item[-30:] for item in TSX_files]
TSX_date1 = [file[0:8] for file in file_list]
TSX_date2 = [file[10:18] for file in file_list]

print(TSX_date1)
print(TSX_date2)
# "/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/1_day_tiffs/x20201116_x20201127.dint.gc.tif"
TSX_extent_list= []
for TSX_file in TSX_files:
    with rio.open(TSX_file) as img:
        TSX_extent=[img.bounds.left,img.bounds.right,img.bounds.bottom,img.bounds.top]
        TSX_extent_list.append(TSX_extent)
print(TSX_extent_list)

TSX_jpg_files = [file for file in np.sort(glob.glob("/Users/tz20896/Documents/PhD/projects/Merapi2021/TSX/staring_134/1_day_tiffs/*dint_flt.gc.jpg"))]

image_files_TSX = []
for TSX_jpg_file in TSX_jpg_files:
    print(TSX_jpg_file)
    with Image.open(TSX_jpg_file) as img:
        image_files_TSX.append(np.array(img))




['20201116', '20201127', '20201208', '20201219', '20210110', '20210201', '20210223', '20210317', '20210408', '20210430', '20210522', '20210602', '20210613', '20211001']
['20201127', '20201208', '20201219', '20210110', '20210201', '20210223', '20210317', '20210408', '20210430', '20210522', '20210602', '20210613', '20211001', '20211023']
[[110.41333067352406, 110.48004187607516, -7.56558230630975, -7.52300627352405], [110.41333067352406, 110.48010524673846, -7.5662884122135505, -7.52369427352405], [110.41333067352406, 110.47871109214586, -7.56581770630975, -7.52324167352405], [110.41333067352406, 110.48187057235896, -7.56284828269455, -7.5203446735240504], [110.41333067352406, 110.48224174338687, -7.56257672974265, -7.520082173524051], [110.41333067352406, 110.48406138671876, -7.562458976790751, -7.51997347352405], [110.41333067352406, 110.48610735384815, -7.562793976790751, -7.52030847352405], [110.41333067352406, 110.48407043967066, -7.56335522974265, -7.520860673524051], [110.41333067

In [22]:
%matplotlib osx
import string
import matplotlib.patheffects as path_effects

plt.close('all')
## plot results
## 28 ifgs (not all are helpfull ignore first 8? --> 20 left --> 5x4 grid)

fig = plt.figure(figsize=(8,8))

gs = GridSpec(4, 4, figure=fig)

# allocating panels in gridspec
ax00 = fig.add_subplot(gs[0, 0]) # 
ax01 = fig.add_subplot(gs[0, 1]) # 
ax02 = fig.add_subplot(gs[0, 2]) # 
ax03 = fig.add_subplot(gs[0, 3]) # 
# ax04 = fig.add_subplot(gs[0, 4]) # 

ax10 = fig.add_subplot(gs[1, 0]) # 
ax11 = fig.add_subplot(gs[1, 1]) # 
ax12 = fig.add_subplot(gs[1, 2]) # 
ax13 = fig.add_subplot(gs[1, 3]) # 
# ax14 = fig.add_subplot(gs[1, 4]) # 

ax20 = fig.add_subplot(gs[2, 0]) # 
ax21 = fig.add_subplot(gs[2, 1]) # 
ax22 = fig.add_subplot(gs[2, 2]) # 
ax23 = fig.add_subplot(gs[2, 3]) # 
# ax24 = fig.add_subplot(gs[2, 4]) # 

ax30 = fig.add_subplot(gs[3, 0]) # 
ax31 = fig.add_subplot(gs[3, 1]) # 
# ax32 = fig.add_subplot(gs[3, 2]) # 
# ax33 = fig.add_subplot(gs[3, 3]) # 
# ax34 = fig.add_subplot(gs[3, 4]) # 


## plot ifgs
lon_lims = [110.423, 110.45]
lat_lims = [-7.55, -7.533]
labels_annot = string.ascii_uppercase[0:len(fig.axes)]
bbox = dict(fc="1.0",alpha=0.5)
for i, (ax,ifg,lab_annot,TSX_ext,d1,d2) in enumerate(zip(fig.axes,image_files_TSX,labels_annot,TSX_extent_list,TSX_date1,TSX_date2)):
    print(i)
    # if i==0:
    #     ax.imshow(ifg[::10,::10],extent=TSX_ext)
    # elif i==4:
    #     ax.imshow(ifg[::10,::10],extent=TSX_ext)
    # else:
    ax.imshow(ifg[::2,::2],extent=TSX_ext)
    ax.plot(coords_L1888[:,0]  ,coords_L1888[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1888')
    ax.plot(coords_L1948[:,0]  ,coords_L1948[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1948')
    ax.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='white',linestyle='--',label='L1956')
    ax.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='white',linestyle='--',)
    ax.plot(coords_L1992[:,0]  ,coords_L1992[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1992')
    ax.plot(coords_L1997[:,0]  ,coords_L1997[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1997')
    ax.plot(coords_L1998[:,0]  ,coords_L1998[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1998')
    ax.plot(coords_L1986[:,0]  ,coords_L1986[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1986')
    ax.plot(coords_L1984[:,0]  ,coords_L1984[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1984')
    ax.plot(coords_L1931[:,0]  ,coords_L1931[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1931')
    ax.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')

    ax.annotate(lab_annot,xy=[0.02,0.90],xycoords='axes fraction',fontsize=16, bbox=bbox)
    text = ax.annotate(f'{d1}-\n{d2}',xy=[0.02,0.10],xycoords='axes fraction',fontsize=12,color='white',weight='bold')

    text.set_path_effects([path_effects.Stroke(linewidth=2, foreground='black'),
                       path_effects.Normal()])
    
    ax.set_xlim(lon_lims)
    ax.set_ylim(lat_lims)
    ax.set_aspect('equal', 'box')
    ax.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower right'))
    ax.set_axis_off()

fig.tight_layout()

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [14]:
fig, ax = plt.subplots(1,1, figsize=(8,8))

ax.imshow(image_files_TSX[1],extent=TSX_extent_list[1])
ax.plot(coords_L1888[:,0]  ,coords_L1888[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1888')
ax.plot(coords_L1948[:,0]  ,coords_L1948[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1948')
ax.plot(coords_L1956_1[:,0],coords_L1956_1[:,1],linewidth=2,color='white',linestyle='--',label='L1956')
ax.plot(coords_L1956_2[:,0],coords_L1956_2[:,1],linewidth=2,color='white',linestyle='--',)
ax.plot(coords_L1992[:,0]  ,coords_L1992[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1992')
ax.plot(coords_L1997[:,0]  ,coords_L1997[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1997')
ax.plot(coords_L1998[:,0]  ,coords_L1998[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1998')
ax.plot(coords_L1986[:,0]  ,coords_L1986[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1986')
ax.plot(coords_L1984[:,0]  ,coords_L1984[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1984')
ax.plot(coords_L1931[:,0]  ,coords_L1931[:,1]  ,linewidth=2,color='white',linestyle='--',label='L1931')
ax.plot(coords_CRATER[:,0],coords_CRATER[:,1],linewidth=2,color='black',linestyle='--',label='Crater rim')
ax.set_xlim(lon_lims)
ax.set_ylim(lat_lims)
ax.set_aspect('equal', 'box')
ax.add_artist(ScaleBar(sm.plot.get_1deg_dist(),location='lower right'))
ax.set_axis_off()
